In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [55]:
%load_ext nb_black
import numpy as np
import sys
import os
import re
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['figure.facecolor']='w'
matplotlib.rcParams['figure.autolayout']=True 
matplotlib.rcParams['figure.figsize'] = (10,5)

from scipy.stats import binom, beta, multivariate_normal
import scipy.integrate as integrate

import ipywidgets as widgets
from ipywidgets import interact


%matplotlib widget

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [3]:
# %%html
# <iframe src="http://www.infinitecuriosity.org/vizgp/" width="1200" height="1000"></iframe>

<IPython.core.display.Javascript object>

# Bayesian inferance 
## Combining MCMC and surrogate models

### Stéphane Nilsson


## Bayes' theorem

* $P(\theta | y)= \frac{P(y|\theta)}{P(y)}$
  * $y$: Observed data
  * $\theta$: Parameters


* $P(\theta)  :$ Prior probability
* $P(y|\theta):$ Likelihood
* $P(y)       :$ Evidence
* $P(\theta|y):$ Posterior probability -> What we are interested in


## Coin flip exemple

* We want to evaluate the probability $p$ of getting a head ($\mathrm{H}$)
* We start by assuming we have no prior information on the coin, thus we set the prior $P(p)$=Beta(1,1) (Equivalent to Uniform distribution)
* The likelihood is given by $P(\mathrm{H}|p)$=Binomial($\mathrm{H}$, $\mathrm{H+T}$, $p$)
* Finally, we get the posterior $P(p|H)$=Beta(1,1)xBinomial($\mathrm{H}$, $\mathrm{H+T}$, $p$)

For an analytical derivation of the solution, visit [Bayesian Coin Flip](https://nbviewer.org/github/lambdafu/notebook/blob/master/math/Bayesian%20Coin%20Flip.ipynb)

For more in-depth explanations with Python code, I recommand [Bayesian Methods for Hackers](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers)

In [166]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))


def posterior(H, T, a, b, x):
    y = prior(a, b, x) * likelihood(H, T, x)
    return y / integrate.simpson(y, x)


def likelihood(H, T, x):
    #Normalize likelihood to make it look nicer
    y = binom.pmf(H, H + T, [x]).reshape(-1,)
    return y / integrate.simpson(
        y, x
    )


def prior(a, b, x):
    return beta.pdf(x, a, b)


@interact(
    H=widgets.IntSlider(
        min=0,
        max=50,
        value=0,
        description="Number of heads: ",
        style={"description_width": "initial"},
    ),
    T=widgets.IntSlider(
        min=0,
        max=50,
        value=0,
        description="Number of tails:   ",
        style={"description_width": "initial"},
    ),
    a=widgets.IntSlider(min=1, max=10, value=1, description="a: "),
    b=widgets.IntSlider(min=1, max=10, value=1, description="b: "),
)
def update(H, T, a, b):
    # [l.remove() for l in ax.lines]
    # [l.remove() for l in ax.lines]
    # Clear axis
    ax.cla()
    #Check if ax.collections is empty
    # if len(ax.collections):
    #     ax.collections.pop()
    x = np.linspace(0, 1, 200)

    y_prior = prior(a, b, x)
    y_likelihood = likelihood(H, T, x)
    y_post = posterior(H, T, a, b, x)

    
    ax.plot(x, y_prior, "--", c="blue", label="Prior",alpha=0.6)
    ax.plot(x, y_likelihood, "--", c="red", label="Likelihood",alpha=0.6)
    ax.plot(x, y_post, c="violet",label="Posterior")
    ax.fill_between(x, x*0, y_post, color="violet",alpha=0.2)
    
    ax.set_ylim(
        (np.min(y_post) - 1e-1, np.max(np.r_[y_post, y_likelihood, y_prior]) + 1e-1)
    )
    ax.set_xlim((0, 1))

    plt.xlabel('$p$')
    plt.ylabel('PDF')
    plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Number of heads: ', max=50, style=SliderStyle(descriptio…

<IPython.core.display.Javascript object>

## What if we cannot compute directly $P(\theta|y)$?
### => MCMC 

Algorithm :
1. Initialize with an arbitrary set of parameters $\theta$ and set a probability distribution function from which the next candidate $\theta'$ will be picked from. Usually it is a Gaussian centered around $\theta$.
2. Randomly pick a new candidate $\theta'$ following the distribution.
3. Compute the acceptance probability $\alpha=\frac{P(y|\theta')P(\theta')}{P(y|\theta')P(\theta)}$
4. Accept or reject :
  * Draw a random number $u \in [0,1]$
  * If $\alpha>u$ , accept and set $\theta_{t+1}=\theta'$
  * If $\alpha<u$ , reject and set $\theta_{t+1}=\theta$

For more information about different sampling techniques, visit [MCMC Algorithms](https://m-clark.github.io/docs/ld_mcmc/index_onepage.html) 

Now let's apply the algorithm to the previous example and compare to analytical solution as the number of samples grows.

This time we assume that the coin is biased towards Heads, and thus set the prior distribution to Beta(10,5).

We flip the coin 50 times and obtain 28 $\mathrm{H}$

In [144]:
def MCMC(n_samples,accepted,rejected):
    a, b=10, 5
    H, T=28, 22
    

    #Sample first sample from prior
    p_old=beta.rvs(a=10,b=5,size=1)
    for _ in range(n_samples):
        #Sample new sample from Gaussian distribution centered around p_old
        p_new=multivariate_normal.rvs(mean=p_old,cov=0.1)
        #Redraw if probability not between 0 and 1
        while not 0<p_new<1:
            p_new=multivariate_normal.rvs(mean=p_old,cov=0.1)


        likelihood= lambda p: binom.pmf(H, H + T, p).reshape(-1,)
        prior= lambda p: beta.pdf(p, a, b)

        alpha=likelihood(p_new)*prior(p_new)/(likelihood(p_old)*prior(p_old))
        
        u=np.random.random(1)
        if alpha>u:
            accepted.append(p_new)
            p_old=p_new
        else:
            rejected.append(p_new)
    
    return accepted, rejected
    

<IPython.core.display.Javascript object>

In [175]:
from IPython.display import display
import functools

fig, ax=plt.subplots(1,1)
accepted=[]
rejected=[]

x=np.linspace(0,1,100)
y_analytic = posterior(28,22,10,5,x)

dropdown=widgets.Dropdown(
    options=[(1000, 1000), (2000, 2000), (5000, 5000)],
    value=1000,
    description='Number of samples: ',
    style={"description_width": "initial"}
)

button = widgets.Button(description="Draw samples")
output = widgets.Output()

display(dropdown,button, output)

def update(n_samples, accepted, rejected):
    with output:
        accepted, rejected = MCMC(n_samples, accepted, rejected)
        ax.cla()
        ax.hist(accepted,bins=50,density=True, histtype='step', color='blue', label='Drawn samples');
        ax.plot(x,y_analytic, label='Analytical posterior', color='violet')
        ax.fill_between(x, x*0, y_analytic, color="violet",alpha=0.2)
        plt.legend()
        plt.xlabel('$p$')
        plt.ylabel('PDF')

def on_button_clicked(b, n_samples, accepted, rejected):
    update(n_samples, accepted, rejected)


button.on_click(functools.partial(on_button_clicked, n_samples=dropdown.value, accepted=accepted, rejected=rejected))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dropdown(description='Number of samples: ', options=((1000, 1000), (2000, 2000), (5000, 5000)), style=Descript…

Button(description='Draw samples', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>